# Lab 03 - Introduction to Natural Language Processing 01

## FastText

### Question 1

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset

In [ ]:
dataset_shuffle = dataset.shuffle(seed=42, load_from_cache_file=False)

In [ ]:
import fasttext

In [ ]:
from lib.util import lower_and_remove_punctuation
from datasets.dataset_dict import DatasetDict

def format_for_fasttext(dataset : DatasetDict, file_name : str):
    """TODO
    """
    with open(file_name, 'w', encoding="utf-8") as f:
        def write_to_file(element : dict) -> dict:
            """TODO
            """
            f.write("__label__" + str(element['label']) + " " +  lower_and_remove_punctuation(element)['text'] + '\n')
            return element

        dataset.map(write_to_file, load_from_cache_file=False)

In [ ]:
format_for_fasttext(dataset_shuffle['train'], "train.txt")
format_for_fasttext(dataset_shuffle['test'], "test.txt")

### Question 2

In [ ]:
model = fasttext.train_supervised("train.txt")
model

In [ ]:
model.test_label("test.txt")

### Question 3

In [ ]:
# split the train dataset into a train split (80%) et a validation split (20%)
# both have the same proportion of 'label' values
split_dataset = dataset_shuffle["train"].train_test_split(train_size=0.8, seed=42, stratify_by_column='label')
# Add the "validation" split to our `dataset_shuffle`
dataset_shuffle["validation"] = split_dataset.pop("test")
# Update the "train" set to our `dataset_shuffle`
dataset_shuffle["train"] = split_dataset.pop("train")

In [ ]:
format_for_fasttext(dataset_shuffle['train'], "train.txt")
format_for_fasttext(dataset_shuffle['validation'], "validation.txt")

In [ ]:
model_autotuned = fasttext.train_supervised(input='train.txt', autotuneValidationFile='validation.txt', verbose=3)

In [ ]:
model_autotuned.test_label("test.txt")

### Question 4

In [ ]:
print("model dimension:", model.get_dimension())
print("model_autotuned dimension:", model_autotuned.get_dimension())

In [ ]:
print("model input:", model.get_input_matrix().shape)
print("model_autotuned input:", model_autotuned.get_input_matrix().shape)

In [ ]:
print("model output:", model.get_output_matrix().shape)
print("model_autotuned output:", model_autotuned.get_output_matrix().shape)

### Question 5

In [ ]:
import numpy as np
test_data = dataset['test'].map(lower_and_remove_punctuation, load_from_cache_file=False)

In [ ]:
# predict all test dataset
predicted_value = model.predict(test_data['text'])

# cast string labels to int labels
predicted_labels = list(map(lambda x : int(x[0][-1]), predicted_value[0]))

# get all failures index
failures_index_list = np.array(predicted_labels) != np.array(test_data['label'])

# get all failures' text
failures = np.array(test_data['text'])[failures_index_list]
# get all failures' label
failures_labels = np.array(test_data['label'])[failures_index_list]
# get all failures' predicted value
failures_predicted_labels = np.array(predicted_labels)[failures_index_list]

In [ ]:
false_negatif = failures_labels == 1
print("false negatif:\n", failures[false_negatif][0])

In [ ]:
false_positif = failures_labels == 0
print("false positif:\n", failures[false_positif][0])